# import dataset
### create 2 different lists:
     - one with the text and the SNOMED ID
     - secon one with a dictionary of the SNOMED ID and correct label, maybe with the note dicttionary offered by cometa?
     

code inspiration from explosion : https://github.com/explosion/projects/blob/master/nel-emerson/scripts/notebook_video.ipynb 

In [2]:
import pandas as pd

In [11]:
import pandas as pd
original = pd.read_csv("cometa.csv")
original['SNOMED_ID'] = original['SNOMED_ID'].astype(str)
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20015 entries, 0 to 20014
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    20015 non-null  int64 
 1   Term          20015 non-null  object
 2   SNOMED_Label  20015 non-null  object
 3   SNOMED_ID     20015 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.6+ KB


In [94]:
section = original[[
    'Term',
    'SNOMED_ID'
    
]][20:40]

#COMETA
gold_ids = section.SNOMED_ID.values.tolist()
gold_ids = [str(x) for x in gold_ids]

from collections import Counter
print(Counter(gold_ids))

Counter({'182443006': 6, '722482005': 4, '43396009': 3, '734579009': 3, '372733002': 3, '372720008': 1})


In [95]:
#CREATE DICCTIONARY 

#select the collumsn
dict_labelid = original[[
    'SNOMED_ID',
    'SNOMED_Label'
    
]][20:40]

#convert to dictionaty
name_dict = dict(dict_labelid.values)
print("Dict: ", name_dict)

#get the keys
qids = name_dict.keys()
print( "keys: ", qids)

Dict:  {'722482005': 'Doxorubicin, bleomycin, vinblastine and dacarbazine chemotherapy regimen', '43396009': 'Hemoglobin A1c measurement', '734579009': 'Angiotensin-converting enzyme inhibitor', '372733002': 'angiotensin converting enzyme inhibitor', '182443006': 'Entire anterior cruciate ligament of knee joint', '372720008': 'Antidepressant'}
keys:  dict_keys(['722482005', '43396009', '734579009', '372733002', '182443006', '372720008'])


In [96]:
dataset = []

for index, row in section.iterrows():
    example = row
    text = example["Term"]
    offset = 0
    QID = example["SNOMED_ID"]
    links_dict = {QID: 1.0}
    dataset.append((text, {"links": {offset: links_dict}}))
    

In [123]:
dataset[0]

('ABVD', {'links': {0: {'722482005': 1.0}}})

In [120]:
import random


train_dataset = []
test_dataset  = []

for QID in qids:
    
    indices = [i for i, j in enumerate(gold_ids) if j == QID]
    print(indices)

    train_dataset.extend(dataset[index] for index in indices[0:3])  # first 8 in training
    test_dataset.extend(dataset[index] for index in indices[-2:-1])  # last 2 in test
    
random.shuffle(train_dataset)
random.shuffle(test_dataset)



[0, 1, 2, 3]
[4, 5, 6]
[7, 8, 9]
[10, 11, 12]
[13, 14, 15, 16, 17, 18]
[19]


In [121]:
train_dataset

[('ACE inhibitors', {'links': {0: {'372733002': 1.0}}}),
 ('ABVD', {'links': {0: {'722482005': 1.0}}}),
 ('ACL', {'links': {0: {'182443006': 1.0}}}),
 ('ACE inhibitor', {'links': {0: {'734579009': 1.0}}}),
 ('ACE inhibitors', {'links': {0: {'372733002': 1.0}}}),
 ('ACE inhibitor', {'links': {0: {'734579009': 1.0}}}),
 ('ABVD chemo', {'links': {0: {'722482005': 1.0}}}),
 ('ACE inhibitor', {'links': {0: {'734579009': 1.0}}}),
 ('ACL', {'links': {0: {'182443006': 1.0}}}),
 ('AD', {'links': {0: {'372720008': 1.0}}}),
 ('AC1', {'links': {0: {'43396009': 1.0}}}),
 ('ACE inhibitors', {'links': {0: {'372733002': 1.0}}}),
 ('ABVD chemo', {'links': {0: {'722482005': 1.0}}}),
 ('ACL', {'links': {0: {'182443006': 1.0}}}),
 ('AC1', {'links': {0: {'43396009': 1.0}}}),
 ('AC1', {'links': {0: {'43396009': 1.0}}})]

In [122]:
test_dataset

[('ABVD chemo', {'links': {0: {'722482005': 1.0}}}),
 ('ACE inhibitors', {'links': {0: {'372733002': 1.0}}}),
 ('ACL', {'links': {0: {'182443006': 1.0}}}),
 ('AC1', {'links': {0: {'43396009': 1.0}}}),
 ('ACE inhibitor', {'links': {0: {'734579009': 1.0}}})]

In [132]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [133]:
TRAIN_DOCS = []
for text, annotation in train_dataset:
    #print(text, annotation)
    doc = nlp(text)     # to make this more efficient, you can use nlp.pipe() just once for all the texts
    TRAIN_DOCS.append((doc, annotation))

In [134]:
TRAIN_DOCS

[(ACE inhibitors, {'links': {0: {'372733002': 1.0}}}),
 (ABVD, {'links': {0: {'722482005': 1.0}}}),
 (ACL, {'links': {0: {'182443006': 1.0}}}),
 (ACE inhibitor, {'links': {0: {'734579009': 1.0}}}),
 (ACE inhibitors, {'links': {0: {'372733002': 1.0}}}),
 (ACE inhibitor, {'links': {0: {'734579009': 1.0}}}),
 (ABVD chemo, {'links': {0: {'722482005': 1.0}}}),
 (ACE inhibitor, {'links': {0: {'734579009': 1.0}}}),
 (ACL, {'links': {0: {'182443006': 1.0}}}),
 (AD, {'links': {0: {'372720008': 1.0}}}),
 (AC1, {'links': {0: {'43396009': 1.0}}}),
 (ACE inhibitors, {'links': {0: {'372733002': 1.0}}}),
 (ABVD chemo, {'links': {0: {'722482005': 1.0}}}),
 (ACL, {'links': {0: {'182443006': 1.0}}}),
 (AC1, {'links': {0: {'43396009': 1.0}}}),
 (AC1, {'links': {0: {'43396009': 1.0}}})]

# CREATE KNOWLEDGE GRAPH


In [159]:
from pathlib import Path
def load_entities():
    entities_loc = Path.cwd() / "cometa.csv"  # distributed alongside this notebook

    names = dict()
    descriptions = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        next(csvreader, None)  # skip the headers
        for row in csvreader:
            qid = row[3]
            #print(qid)
            name = row[1]
            desc = row[2]
            names[qid] = name
            descriptions[qid] = desc
    return names, descriptions

In [160]:
name_dict, desc_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name={name_dict[QID]}, desc={desc_dict[QID]}")

387172005, name=5FU, desc=Fluorouracil
73916008, name=HTP, desc=Oxitriptan
386835005, name=purinethol, desc=Mercaptopurine
443911005, name=A1C, desc=Ordinal level of hemoglobin A1c
43396009, name=HbA1c test, desc=Hemoglobin A1c measurement
733830002, name=hb1ac, desc=Glycated hemoglobin-A1c
722482005, name=ABVD chemo, desc=Doxorubicin, bleomycin, vinblastine and dacarbazine chemotherapy regimen
734579009, name=ACE inhibitor, desc=Angiotensin-converting enzyme inhibitor
372733002, name=ACE inhibitors, desc=angiotensin converting enzyme inhibitor
182443006, name=ACL, desc=Entire anterior cruciate ligament of knee joint
372720008, name=antidepressants, desc=Antidepressant
30065008, name=ADA, desc=Adenosine deaminase
35253001, name=difficulty concentrating, desc=Attention deficit hyperactivity disorder, predominantly inattentive type
406506008, name=adhd, desc=Attention deficit hyperactivity disorder
702538008, name=ADHD meds, desc=Drug therapy for attention deficit hyperactivity disorder


76948002, name=searing pain, desc=Severe pain
387221005, name=cromolyn sodium, desc=Cromoglicate sodium
26893007, name=groin, desc=groin
69640009, name=scabs, desc=Crust
95668009, name=facial pain, desc=Pain in face
28263002, name=feel like crying, desc=Crying
26782000, name=cryotherapy, desc=Cryotherapy
769069006, name=crystals, desc=Crystal
56177003, name=cubital tunnel syndrome, desc=Cubital tunnel syndrome
227392001, name=cumin, desc=Cumin seed
89028002, name=tumeric, desc=Curcumin stain
6620001, name=cuticle, desc=Structure of cuticle of nail
259102005, name=cyanide, desc=Cyanide
76105009, name=cyclothymia, desc=Cyclothymia
372683000, name=cyproheptadine, desc=Cyproheptadine
367643001, name=cyst, desc=Cyst
13277001, name=cystic acne, desc=Cystic acne
24139008, name=cystoscopy, desc=Endoscopy of urinary bladder
75777003, name=cytokines, desc=Cytokine
258771000, name=dL, desc=Deciliter
200849007, name=daith, desc=Piercing of external ear
57182000, name=neurological damage, desc=nerv

275322007, name=scarred, desc=Scar
46991000, name=scatterbrained, desc=Absent minded
442111003, name=scd, desc=Intermittent pneumatic compression stockings
5987007, name=scents, desc=Recognition odor
58214004, name=schizophrenics, desc=Schizophrenia
68890003, name=schizoaffective disorder, desc=Schizoaffective disorder
23056005, name=sciatica, desc=Sciatica
41519007, name=sciatic nerve, desc=Structure of sciatic nerve
2228002, name=scintillating scotoma, desc=Scintillating scotoma
64973003, name=scissors, desc=Scissors, device
89155008, name=scleroderma, desc=Systemic sclerosis
298382003, name=scoliosis, desc=Scoliosis deformity of spine
400157005, name=scratches, desc=Scratch
360156006, name=screenings, desc=Screening - procedure intent
20233005, name=scrotum, desc=Scrotal structure
311899008, name=sealants, desc=Sealant
444316004, name=seasonal allergies, desc=Seasonal allergy
417992006, name=sebaceous cyst, desc=Sebaceous cyst
50563003, name=seborrheic dermatitis, desc=Seborrheic de

In [162]:
from spacy.kb import KnowledgeBase
nlp = spacy.load("en_core_web_lg")
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

In [163]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here

In [164]:
for qid, name in name_dict.items():
    print(qid,name)
    kb.add_alias(alias=name, entities=[qid], probabilities=[1]) 

387172005 5FU
73916008 HTP
386835005 purinethol
443911005 A1C
43396009 HbA1c test
733830002 hb1ac
722482005 ABVD chemo
734579009 ACE inhibitor
372733002 ACE inhibitors
182443006 ACL
372720008 antidepressants
30065008 ADA
35253001 difficulty concentrating
406506008 adhd
702538008 ADHD meds
49436004 atrial fibrillation
774099003 AHI
62479008 AIDs
259496005 ALA
86044005 ALS
56935002 ALT
91861009 Acute Myeloid Leukemia
45530001 ANA
359788000 ANA test
26843008 antiphospholipid syndrome
387501005 mesalazine
26665006 sociopathy
26091008 AST
76711004 ATP
397758007 AVN
372574004 immuran
257729003 Ablations
426653008 Abraxane
128477000 abscesses
387517004 paracetamol
61352006 Achilles tendon
386938006 acitretin
444648007 Actemra
395828009 Actos
96367001 Afrezza
91857003 Acute Lymphoblastic Leukemia
407317001 humira
50868004 Adderall
363732003 adrenal insufficiency
784314006 Adenomyosis
817000 Adhesive
387362001 epinephrine
372817009 doxorubicin
772470004 Advair
387207008 ibuprophen
387158001 Afr

410502007 JIA
410795001 JRA
109032009 Jadelle
427522005 Janumet
423307000 Januvia
703894008 Jardiance
661005 jaws
412256008 Taytulla
22606007 Vitamin K2
765060000 ketogenic
372572000 Viagra
387304003 cephalexin
373464007 Ketamine
333847008 ketamine infusions
372642003 Ketotifen
716125002 Pembrolizumab
95570007 kidney stones
19063003 knee surgery
89321004 Koebner phenomenon
14293000 L2
36470004 L3
181844007 L4
181845008 L5
75782005 L5 - S1
425585009 L5 / S1
426875007 lada
312965008 lasik
61745006 LCHF
259319003 LDH
22244007 LDL
102739008 LDL cholesterol
36899001 LEEP procedure
91343008 LES
64182005 LH
773127003 LINX surgery
106292003 LPN
414581006 silent reflux
25674000 LS
15698006 LSD
47703008 Lactose
387562000 lamotrigine
373450007 terbinafine
350810002 lancet
386888004 Prevacid
776343005 Lantus
768330004 Larin
703115008 Latuda
61685007 legs
93143009 leukemias
387552007 Levofloxacin
414515005 Levemir
414518007 Levimir
326324002 Microgynon
372866006 Librium
387480006 lidocaine
77586900

/opt/anaconda3/lib/python3.8/runpy.py:193: UserWarning: [W017] Alias 'T3' already exists in the Knowledge Base.
  return _run_code(code, main_globals, None,


73187006 thyroxine
56717001 tuberculosis
127295002 traumatic brain injury
33219003 amitryptaline
373253007 tricyclics
229559001 TENS
700669005 TENS machine
272393004 tests
72024007 THC
266257000 TIAs
41888000 TMJ disorder
15639721000119105 TMJ pain
386207004 TMJD
704956003 TMS
31681005 trigeminal neuralgia
706970001 TNBC
8612007 TNF
128210009 Thoracic Outlet Syndrome
225372007 TPN
724163007 TRT
65428006 TSH
18504008 TSS
169449001 TTC
373541007 cimetidine
724037000 Taltz
412261005 Tamiflu
373345002 tamoxifen
703129009 Tanzeum
414123001 Tarceva
786204003 Tarina
81634008 Tarlov cysts
399014008 Tdap
317317009 Tecta
387009002 temozolomide
43688007 testosterone
85600001 trigs
49928004 throats
76505004 Thumbs
289940003 Tiger Balm
701962007 Timesulin
373440006 tizanidine
21974007 tongue
777808008 Topamax
768759001 steroid ointment
386844006 topirimate
5158005 Tourettes
702798009 Tradjenta
417746004 traumas
736494008 Tremfya
710281005 Tresiba
768374002 Treximet
387025007 Trileptal
714080005 Tru

702772003 chemical sensitivity
441900009 chemicals
265760000 chemo infusion
266719004 chemo pills
386806002 cognitive problems
703423002 chemoradiation
51185008 chest
43799004 chest cavity
50417007 chest cold
279084009 chest discomfort
29857009 pain in my chest
128601007 lung infection
81802002 chest muscles
23924001 tight chest
399208008 chest xray
38907003 chickenpox
386216000 childbirth
43724002 shivers
30291003 chin
182548004 chiropractics
105629000 chlamydia
50213009 chloride
29276003 chlorine
70179006 cholangiocarcinoma
38102005 cholecystectomy
84698008 cholesterol
65123005 choline
73098005 cholinergic urticaria
4104007 chondroitin
111070004 chromium
91272006 chromosome
111985007 chronic abdominal pain
191708009 chronic anxiety
134407002 chronic back pain
27624003 lifelong disease
90734009 chronic conditions
236069009 chronic constipation
68154008 chronic cough
571000119103 chronic daily headaches
427419006 chronic migraines
192080009 chronic depression
236071009 chronic diarrhea

...

/opt/anaconda3/lib/python3.8/runpy.py:193: UserWarning: [W017] Alias 'hypersensitivity' already exists in the Knowledge Base.
  return _run_code(code, main_globals, None,


limbs
58590008 limbic
11038000 limonene
301371003 persistent pain
48477009 lips
72993008 lipase
70106000 lipids
16254007 lipid panel
238894004 lipohypertrophy
93163002 lipomas
52793004 nitrogen
53527002 liquor
85899009 lithium
10200004 livers
86259008 liver biopsy
93870000 liver cancer
243978007 liver damage
235856003 liver disease
259670004 liver enzymes
59927004 liver failure
26958001 liver tests
300338006 liver problems
94381002 liver mets
107963000 liver resection
197354009 liver toxicity
18027006 liver transplants
125571002 lobectomy
273991000 lobotomy
112104007 localized pain
767939007 lomotil
398032003 watery diarrhea
89362005 weightloss
161832001 losing weight
248096004 loss of libido
309257005 lost a lot of weight
739000003 lotion
166891009 low glucose
419188005 low WBC
264072009 lower back
279039007 low back pain
66539004 low carbs
304213008 mild fever
165397008 low hemoglobin
55961006 low stomach acid
131078003 low testosterone
27033000 lower abdomen
48144002 lower back musc

/opt/anaconda3/lib/python3.8/runpy.py:193: UserWarning: [W017] Alias 'pain tolerance' already exists in the Knowledge Base.
  return _run_code(code, main_globals, None,


physical problems
417163006 physical damage
256235009 physical exertion
272144002 sensations
55539008 physical stress
722138006 physical therapy
91251008 physiotherapy
46992007 pills
47639008 pilonidal cyst
103605005 zits
285305004 pimples
45793000 pineal gland
12406000 pinky finger
62507009 tingly
129179000 piriformis syndrome
75699001 piriformis muscle
227512001 pistachios
56329008 pituitary gland
34630004 pityriasis
182886004 placebos
78067005 placenta
200965009 plaque psoriasis
1522000 plaques
50863008 plasma
72191006 plasma glucose
394611003 plastic surgery
16378004 platelets
84847000 platinum
196075003 pleurisy
233604007 pneumonia
398102009 polio
300910009 pollen allergy
256259004 pollens
781067001 polycystic ovaries
441456002 polyp
41329004 polyps
774731005 polysporin
88480006 potassium
74480006 potassium supplements
426508001 pouchitis
777310000 prednisone
77386006 preg
94641000119109 pregnancy anxiety
74036000 pregnancy tests
44001008 premature ejaculation
398193008 premeds
40

5447007 transfusions
704954000 transmitter
77465005 transplants
13205008 vaginal ultrasound
31764008 trapezius
372829000 trazodone
277132007 treatments
387305002 tretinoin
181623009 triceps
56335008 trichomoniasis
17155009 trichotillomania
27612005 trigeminal nerve
45906009 triphasic
102680009 troponin
130616001 tryptase
54821000 tryptophan
77543007 tubal ligation
83059008 tubes
706842002 tubing
108369006 tumours
420990001 tunnel vision
46607005 turbinates
227414003 turmeric
249966004 twitchy
14092002 tyramine
27378009 tyrosine
56208002 ulcers
261182007 ulnar
58696008 ulnar nerve
301726009 unable to function
224794009 uncaring
439420004 uncircumcised
248342006 underweight
4481000175102 undiagnosed diabetes
225779001 undigested food
135818000 unfit
26329005 unfocused
45763006 unnerving
422768004 unresponsive
103361006 unsteady
279026008 upper abdomen
69273007 upper arms
264240001 upper back
128262006 upper body
703712000 upper butt
264242009 upper chest
4335006 upper jaw
11681001 upper 

In [173]:
# ?????????????/
qids = name_dict.keys()
probs = [0.3 for qid in qids]
kb.add_alias(alias="Emerson", entities=qids, probabilities=probs)

ValueError: [E133] The sum of prior probabilities for alias 'Emerson' should not exceed 1, but found 1093.4999999999256.

In [165]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['75540009', '225586007', '241705007', '258158006', '398714009', '312230002', '57019003', '67290009', '297982009', '41329004', '11766001', '107578002', '372688009', '68890003', '31528005', '78702007', '43116000', '31684002', '267064002', '79706000', '84619001', '39723000', '203095000', '117253005', '255635008', '387442005', '87770006', '66760008', '68887009', '73238002', '53430007', '165232002', '300913006', '39423001', '427419006', '34486009', '119292006', '387172005', '372670001', '224994002', '302440009', '32914008', '224978009', '387222003', '395781005', '423121009', '162011005', '236071009', '363346000', '125594001', '76067001', '241686001', '9826008', '18027006', '52954000', '4830009', '248255005', '12611008', '16254007', '706507009', '42399005', '275498002', '284523002', '65183007', '7885001', '7973008', '44609006', '68498002', '409516001', '75665004', '169314007', '72405004', '715178005', '782415009', '11681001', '16650009', '55406008', '784617008', '2981810

In [172]:
print(f"Candidates for 'Roy Stanley Emerson': {[c.entity_ for c in kb.get_candidates('rosacea')]}")
print(f"Candidates for 'Emerson': {[c.entity_ for c in kb.get_candidates('stomach flu')]}")
print(f"Candidates for 'Sofie': {[c.entity_ for c in kb.get_candidates('5FU')]}")

Candidates for 'Roy Stanley Emerson': ['398909004']
Candidates for 'Emerson': ['25374005']
Candidates for 'Sofie': ['387172005']
